# 6. Data Manipulation IV - Combining Data

The goal of this module is to learn to perform Polars queries that involve combining data. More specifically, we'll cover two new Query Statements:
1. Joining dataframes with `.join()`.
2. Concatenating dataframes with `.concat()`.

But first we import `polars`...

In [1]:
import polars as pl

... and load the data, this time changing the name from `df` to `march_yellow_rides_df` for what's in store in this module.

In [2]:
yellow_rides_column_rename_mapping = {
    "VendorID": "vendor_id",
    "RatecodeID": "ratecode_id",
    "PULocationID": "pu_location_id",
    "DOLocationID": "do_location_id",
    "Airport_fee": "airport_fee",
}
march_yellow_rides_df = (
    pl.read_parquet("../data/yellow_tripdata_2024-03.parquet")
    .rename(yellow_rides_column_rename_mapping)
)

## 5.1. Joining Dataframes

In the previous module, we were interested in seeing some summary statistics about each of the different pickup location IDs:

In [3]:
location_id_summary_df = (
    march_yellow_rides_df
    .group_by("pu_location_id")
    .agg(
        pl.len().alias("count_trips"),
        pl.col("trip_distance").min().name.suffix("_min"),
        pl.col("trip_distance").max().name.suffix("_max"),
        pl.col("trip_distance").mean().name.suffix("_mean"),
    )
    .sort(pl.col("count_trips"), descending=True)
)
location_id_summary_df.head()

pu_location_id,count_trips,trip_distance_min,trip_distance_max,trip_distance_mean
i32,u32,f64,f64,f64
161,163269,0.0,51066.77,2.692728
132,157706,0.0,9211.95,15.76677
237,155631,0.0,44866.77,2.096025
236,146044,0.0,109619.96,3.481146
162,123805,0.0,57408.32,3.030781


This is nice, but if we try to present these results to any sort of business stakeholder, the first thing they'll say is "What are these location IDs? What is their actual name?" Thankfully, that information is stored in another file, `"taxi_zone_lookup.parquet"`. Let's load it in...

In [14]:
zones_df = pl.read_parquet("../data/taxi_zone_lookup.parquet")
zones_df.head()

LocationID,Borough,Zone,service_zone
i32,str,str,str
1,"""EWR""","""Newark Airport""","""EWR"""
2,"""Queens""","""Jamaica Bay""","""Boro Zone"""
3,"""Bronx""","""Allerton/Pelham Gardens""","""Boro Zone"""
4,"""Manhattan""","""Alphabet City""","""Yellow Zone"""
5,"""Staten Island""","""Arden Heights""","""Boro Zone"""


This dataframe contains information about each location ID, and the `str` names for the Borough, Zone, and service zone that each location ID represents. Let's quickly rename the columns to make them snake_case like those of `march_yellow_rides_df`:

In [15]:
zones_df = (
    zones_df
    .rename({
        "LocationID": "location_id",
        "Borough": "borough",
        "Zone": "zone",
    })
)
zones_df.head()

location_id,borough,zone,service_zone
i32,str,str,str
1,"""EWR""","""Newark Airport""","""EWR"""
2,"""Queens""","""Jamaica Bay""","""Boro Zone"""
3,"""Bronx""","""Allerton/Pelham Gardens""","""Boro Zone"""
4,"""Manhattan""","""Alphabet City""","""Yellow Zone"""
5,"""Staten Island""","""Arden Heights""","""Boro Zone"""


Perfect.

If we want to now combine it with the summary statistics we just computed, we can do so by **joining** it into the summary statistics table, using `pl.DataFrame.join()`:

In [ ]:
(
    location_id_summary_df
    .join(
        zones_df,
        left_on="pu_location_id",
        right_on="location_id"
    )
    .head()
)

Nice! The data from `zones_df` has been joined into our zone-summary table, and we can see the `"borough"`, `"zone"`, and `"service_zone"` alongside the `"pu_location_id"`! `polars`'s `.join()` functionality has all the same functionality you may be familiar with from Pandas and/or SQL:
1. `location_id_summary_df` specifies the **left** table for the join.
2. `.join(...)` starts the join.
3. `zones_df` specifies the **right** table for the join.
4. `left_on` specifies the column to join on from `location_id_summary_df`.
5. `right_on` specifies the column to join on from `zones_df`.

Just to be clear, though, we didn't have to do the aggregation to `location_id_summary_df`, first; we can join directly into the original `march_yellow_rides_df`:

In [ ]:
(
    march_yellow_rides_df
    .select(["tpep_pickup_datetime", "pu_location_id", "do_location_id", "trip_distance", "total_amount"])
    .join(
        zones_df,
        left_on="pu_location_id",
        right_on="location_id",    
    )
#     .select(["tpep_pickup_datetime", "pu_location_id", "do_location_id", "trip_distance", "total_amount", "borough", "zone"])
    .head()
)

But now, it's a bit strange that we have the `"pu_location_id"` and `"do_location_id"` in the same dataframe, because it's not clear whether the `"borough"` and `"zone"` refer to the `"pu_location_id"` or the `"do_location_id"`. We can quickly fix that:

In [ ]:
zone_columns = ["borough", "zone", "service_zone"]
(
    march_yellow_rides_df
    .select(["tpep_pickup_datetime", "pu_location_id", "do_location_id", "trip_distance", "total_amount"])
    .join(
        zones_df,
        left_on="pu_location_id",
        right_on="location_id",
    )
    .with_columns([pl.col(zone_columns).name.prefix("pu_")])
    .drop(zone_columns)
    .head()
)

We can also use the `on` argument in case the name of the two columns is the same in both tables. For example, imagine we wanted to join `zones_df` with the table we created in the prior module, of rides that both started in the same location ID; then we don't need to specify pickup vs dropoff:

In [ ]:
(
    march_yellow_rides_df
    .filter(pl.col("do_location_id").eq(pl.col("pu_location_id")))
    .select([
        "tpep_pickup_datetime",
        pl.col("pu_location_id").alias("location_id"),
        "trip_distance",
        "total_amount",
    ])
    .join(
        zones_df,
        on="location_id",
    )
    .head()
)

`polars` also supports all possible joins: anti joins, left joins, right joins, inner joins, outer joins... you've got it all, all you have to use is the `how` argument. To this end, there's a curious difference between `zones_df` and `march_yellow_rides_df`...

In [ ]:
print(f"Number of distinct pu_location_id in `march_yellow_rides_df`: {march_yellow_rides_df.select(pl.col('pu_location_id').n_unique())}")
print(f"Number of distinct pu_location_id in `zones_df`: {zones_df.select(pl.col('location_id').n_unique())}")

There are exactly **6** more `location_id`s in `zones_df` than there are `pu_location_id`s in `march_yellow_rides_df`! This makes sense--`zones_df` serves as an index of **all** zones, and there's no guarantee that all zones had at least one ride. What if we wanted to see which zones didn't have any rides that started in them (i.e. `location_id`s with no matching `pu_location_id`)? For that, we can use an anti-join:

In [ ]:
(
    zones_df
    .join(march_yellow_rides_df, right_on="pu_location_id", left_on="location_id", how="anti")
)

We can confirm, the result has a length of **6** data points; exactly what we saw in our `n_unique()` check.

And, of course, as with the other Query Statements, we can join not only on a column name `str`, but also on a `pl.Expr` object. Consider the following toy example, where we want to join two columns together based on whether or not they are even or odd:

In [ ]:
left_df = pl.DataFrame({"a": [1, 2, 3, 4], "b": ["Danny", "Donna", "Lana", "Sauna"]})
right_df = pl.DataFrame({"c": [1, 2], "d": ["cool", "not cool"]})
display(left_df)
display(right_df)

joined_df = (
    left_df
    .join(
        right_df,
        left_on=pl.col("a") % 2,
        right_on=pl.col("c") % 2
    )
)
display(joined_df)

Note, `2024-05-30`: At the time of writing, `polars` does not yet support non-equi joins. In other words, the join condition has to be an equality condition, like "dataframe 1 column equals dataframe 2 column". This is less flexible than join options offered by e.g. typical SQL, where you can join on a non-equality condition like "dataframe 1 column is less than dataframe 2 column". But not to fret! `polars` contributors are actively working on this; and you can stay tuned to the progress here: https://github.com/pola-rs/polars/issues/10068.

## 5.2. Concatenating Dataframes

The data we've been working with so far has been the file `"../data/yellow_tripdata_2024-03.parquet"`, which contains all rides given by yellow taxis in NYC in the month of March, 2024. Amazingly, New York City records and publishes this data every month, going all the way back to 2009! What if we wanted to analyze two months' datasets together, in the same dataframe? Well, for that, we have `pl.concat()`.

Let's start by loading in February's yellow taxi trip data. It should have the same schema as the data from March, so we can use the same column name mapping dictionary.

In [ ]:
february_yellow_rides_df = (
    pl.read_parquet("../data/yellow_tripdata_2024-02.parquet")
    .rename(yellow_rides_column_rename_mapping)
)
february_yellow_rides_df.head()

To combine these two dataframes, we just have to pass them in as a list to `pl.concat()`, and we can see the result:

In [ ]:
all_yellow_rides_df = pl.concat([
    february_yellow_rides_df,
    march_yellow_rides_df,
])
display(yellow_rides_df.head())
print(f"{february_yellow_rides_df.shape[0]} rides recorded in `february_yellow_rides_df`.")
print(f"{march_yellow_rides_df.shape[0]} rides recorded in `march_yellow_rides_df`.")
print(f"{all_yellow_rides_df.shape[0]} rides recorded in `all_yellow_rides_df`.")

And just like that, we have a combined dataframe of all the yellow taxi rides from February and March! Yellow taxis aren't the only type of taxis in New York City, though; for example, there are also green taxis, which the city of NYC also records. However, a slightly different taxi type means slightly different data... how will it be to combine such data together? Let's see...

Let's start by loading in `march_green_rides_df` from file:

In [ ]:
march_green_rides_df = pl.read_parquet("../data/green_tripdata_2024-03.parquet")
march_green_rides_df.head()

Already, we can see some slight differences between the `all_yellow_rides_df` and `march_green_rides_df`--for example, `march_green_rides_df` has 20 columns, while we're accustomed to the 19 columns in `march_yellow_rides_df`--but there are also a lot of similarities. For one, `march_green_rides_df` has the same `datetime` columns (though with slighly different names), and the same TitleCase ID columns. We'll need to conform the column names so that the correct columns match up with each other upon concatenation.

In [ ]:
green_rides_df_column_mapping = {
    "VendorID": "vendor_id",
    "RatecodeID": "ratecode_id",
    "PULocationID": "pu_location_id",
    "DOLocationID": "do_location_id",
    "lpep_pickup_datetime": "tpep_pickup_datetime",
    "lpep_dropoff_datetime": "tpep_dropoff_datetime",
    # "Airport_fee": "airport_fee",  # Doesn't exist in `green_rides_df`
}
march_green_rides_df = march_green_rides_df.rename(green_rides_df_column_mapping)

Great! Now, if we want to finally combine the yellow taxi dataframes and green taxi dataframe with `pl.concat()`, we just pass them in as a list again:

In [ ]:
all_rides_df = (
    pl.concat([
        february_yellow_rides_df,
        march_yellow_rides_df,
        march_green_rides_df,
    ])
)

Even with the renaming, we got an error! That's because, by default, the two dataframes have to have the same schema. This isn't the case in our dataframes, though; however we'd still like to concatenate them! Thankfully, `polars` gives us flexible control over this, with the `how` argument. By default, `how` is set to `how="vertical"`, restricting the concatenation to a strict vertical stacking of the dataframes. However, we can set `how="diagonal"`, and the two dataframes will be concatenated together, with nulls used to fill in the spaces of unshared columns:

In [ ]:
all_rides_df = (
    pl.concat(
        [
            february_yellow_rides_df,
            march_yellow_rides_df,
            green_rides_df,
        ],
        how="diagonal"
    )
)
all_rides_df.head()

Scrolling to the right, we can see that `ehail_fee` and `trip_type` are null, since the yellow trips data doesn't have these two columns--they are brought over from `green_rides_df`.

`pl.concat()` does offer a few more options, such as horizontal concatenation instead of vertical concatenation, but it's not so useful for us here; instead, I leave it to your reading!

# Conclusion

In this module, we learned how to combine multiple dataframes together, using two new Query Statements, `.join()` and `.concat()`.